In [1]:
import cv2
import numpy as np

def transparentOverlay(src, overlay, pos=(0, 0), scale=1):
    overlay = cv2.resize(overlay, (0, 0), fx=scale, fy=scale)
    h, w, _ = overlay.shape
    rows, cols, _ = src.shape 
    y, x = pos[0], pos[1] 

    for i in range(h):
        for j in range(w):
            if x + i >= rows or y + j >= cols:
                continue
            alpha = float(overlay[i][j][3] / 255.0)
            src[x + i][y + j] = alpha * overlay[i][j][:3] + (1 - alpha) * src[x + i][y + j]
    return src

faceCascade= cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv2.imread('Jamie_Before.jpg')
specs_ori = cv2.imread('glasses.png',-1)
mus_ori = cv2.imread('mustache.png',-1)
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(imgGray,1.1,4)
 
for (x,y,w,h) in faces:
    
    if h > 0 and w > 0:
        glass_symin = int(y + 1.5 * h / 5)
        glass_symax = int(y + 2.5 * h / 5)
        sh_glass = glass_symax - glass_symin

        mus_symin = int(y + 3.5 * h / 6)
        mus_symax = int(y + 5 * h / 6)
        sh_mus = mus_symax - mus_symin

        face_glass_roi_color = img[glass_symin:glass_symax, x:x + w]
        face_mus_roi_color = img[mus_symin:mus_symax, x:x + w]

        specs = cv2.resize(specs_ori, (w, sh_glass), interpolation=cv2.INTER_CUBIC)
        mustache = cv2.resize(mus_ori, (w, sh_mus), interpolation=cv2.INTER_CUBIC)
        
        transparentOverlay(face_glass_roi_color, specs)
           
        transparentOverlay(face_mus_roi_color, mustache)
cv2.imshow('Final Image',img)
cv2.waitKey()
cv2.destroyAllWindows()